# Transportation Model under changing commodity status

In [2]:
from pulp import *

### Supply nodes (e.g Factories, warehouse) and their capacities

In [3]:
supply_nodes= ['Factory1','Factory2']

### Demand nodes (e.g retail outlets, customers) and their demands

In [4]:
demand_nodes = ['Customer1','Customer2','Customer3']

commodities= ['Commodity1', 'Commodity2']

### Transportation Cost from supply nodes to demands nodes

In [5]:
initial_transportation_costs= {
    ('Factory1', 'Customer1'):30,
    ('Factory1', 'Customer2'):40,
    ('Factory1', 'Customer3'):20,
    ('Factory2', 'Customer1'):50,
    ('Factory2', 'Customer2'):60,
    ('Factory2', 'Customer3'):30,
}

In [6]:
changing_transportation_costs= {
    ('Factory1', 'Customer1'):40,
    ('Factory1', 'Customer2'):50,
    ('Factory1', 'Customer3'):20,
    ('Factory2', 'Customer1'):70,
    ('Factory2', 'Customer2'):40,
    ('Factory2', 'Customer3'):65,
}

### Creating Linear Programming

In [7]:
transport_model= LpProblem(name='Transportation_Model', sense= LpMinimize)


### Decision variable

In [8]:
x= LpVariable.dicts("Shipment", ((i, j) for i in supply_nodes for j in demand_nodes), lowBound=0, cat='Integer')
print(x)

{('Factory1', 'Customer1'): Shipment_('Factory1',_'Customer1'), ('Factory1', 'Customer2'): Shipment_('Factory1',_'Customer2'), ('Factory1', 'Customer3'): Shipment_('Factory1',_'Customer3'), ('Factory2', 'Customer1'): Shipment_('Factory2',_'Customer1'), ('Factory2', 'Customer2'): Shipment_('Factory2',_'Customer2'), ('Factory2', 'Customer3'): Shipment_('Factory2',_'Customer3')}


### Objective function: minimize transportation costs

In [9]:
transport_model+=lpSum(initial_transportation_costs[(i, j)] * x[(i, j)] for i in supply_nodes for j in demand_nodes)

###  Constraints: supply and demand balance

In [10]:
for i in supply_nodes:
    transport_model+=lpSum(x[(i, j)] for j in demand_nodes)<=80
    
for j in demand_nodes:
    transport_model += lpSum(x[(i, j)] for i in supply_nodes)>=35

In [11]:
transport_model.solve()

1

In [12]:
print("Optimal tansportation plan (initial costs): ")
for i in supply_nodes:
    for j in demand_nodes:
        if x[i, j].varValue>0:
            print(f"Ship {x[(i, j)].varValue} units from {i} to {j}");
            


Optimal tansportation plan (initial costs): 
Ship 35.0 units from Factory1 to Customer1
Ship 35.0 units from Factory1 to Customer2
Ship 10.0 units from Factory1 to Customer3
Ship 25.0 units from Factory2 to Customer3


In [13]:
transport_model.objective= lpSum(initial_transportation_costs[(i, j)] * x[(i, j)] for i in supply_nodes for j in demand_nodes)
print(f"Total Cost of Transportation: $ {value(transport_model.objective)}")

Total Cost of Transportation: $ 3400.0


In [14]:
for (i, j), cost in changing_transportation_costs.items():
    initial_transportation_costs[(i, j)]= cost

In [15]:
transport_model.objective= lpSum(initial_transportation_costs[(i, j)] * x[(i, j)] for i in supply_nodes for j in demand_nodes)

In [16]:
transport_model.solve()

1

In [17]:
print("Optimal tansportation plan changing the cost due to commodity: \n")
for i in supply_nodes:
    for j in demand_nodes:
        if x[i, j].varValue>0:
            print(f"Ship {x[(i, j)].varValue} units from {i} to {j}")
            

Optimal tansportation plan changing the cost due to commodity: 

Ship 35.0 units from Factory1 to Customer1
Ship 35.0 units from Factory1 to Customer3
Ship 35.0 units from Factory2 to Customer2


In [18]:
print(f"Total Cost of Transportation: $ {value(transport_model.objective)}")

Total Cost of Transportation: $ 3500.0


# Transportation Model under changing commodity status

In [19]:
from pulp import *

### Supply nodes (e.g Factories, warehouse) and their capacities

In [20]:
supply_nodes= ['Factory1','Factory2']

### Demand nodes (e.g retail outlets, customers) and their demands

In [21]:
demand_nodes = ['Customer1','Customer2','Customer3']

commodities= ['Bread', 'Egg']

### Transportation Cost from supply nodes to demands nodes

In [22]:
initial_shipments= {
    ('Factory1', 'Customer1'):10,
    ('Factory1', 'Customer2'):40,
    ('Factory1', 'Customer3'):20,
    ('Factory2', 'Customer1'):45,
    ('Factory2', 'Customer2'):60,
    ('Factory2', 'Customer3'):35,
}

### Sample data for changing commodity status

In [23]:
commodity_attributes= {
    'Bread': {'perishable':True, 'demand_variability': 0.1},
    'Egg': {'perishable':True, 'demand_variability': 0.2},
}

### Creating Linear Programming

In [24]:
transport_model= LpProblem(name='Transportation_Model', sense= LpMinimize)


### Decision variable

In [25]:
x= LpVariable.dicts("Shipment", ((i, j) for i in supply_nodes for j in demand_nodes), lowBound=0, cat='Integer')

### Objective function: minimize transportation costs

In [26]:
# transport_model+=lpSum(initial_transportation_costs[(i, j)] * x[(i, j)] for i in supply_nodes for j in demand_nodes)
transport_model += lpSum(x[(i, j)] for i in supply_nodes for j in demand_nodes)

###  Constraints: supply and demand balance

In [27]:
for i in supply_nodes:
    transport_model+=lpSum(x[(i, j)] for j in demand_nodes)<=80
    
for j in demand_nodes:
    transport_model += lpSum(x[(i, j)] for i in supply_nodes)>=60

In [28]:
for j in demand_nodes:
    for c in commodities:
        if commodity_attributes[c]['perishable']:
            transport_model+= lpSum(x[(i, j)] for i in supply_nodes)<= commodity_attributes[c]['demand_variability']*100

In [29]:
transport_model.solve()

-1

In [30]:
print("Optimal tansportation plan\n")
for i in supply_nodes:
    for j in demand_nodes:
        if x[i, j].varValue>0:
            print(f"Ship {x[(i, j)].varValue} units from {i} to {j}")
            


Optimal tansportation plan

Ship 60.0 units from Factory1 to Customer2
Ship 60.0 units from Factory2 to Customer1
Ship 60.0 units from Factory2 to Customer3


In [31]:
print(f"Total Cost of Transportation: $ {value(transport_model.objective)}")

Total Cost of Transportation: $ 180.0
